# WCWH Rollout Preparation
Working to get the beacons ready for their next adventure. 

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Calibration Inspection
Looking at the results from the calibration experiments

In [3]:
import os
import sys
sys.path.append('../')

import pandas as pd
pd.set_option('display.max_columns', 200)
import numpy as np

from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import seaborn as sns
from src.visualization import visualize

from src.calibration import calibrate

import math
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data Import

## Carbon Dioxide
From two environments

In [8]:
co2_house = pd.read_csv("../data/interim/co2-linear_model_testhouse-wcwh_s21.csv",index_col=0)
co2_house.head()

,constant,coefficient
beacon,,
1,342.365073,0.821170
2,0.000000,1.000000
3,0.000000,1.000000
4,0.000000,1.000000
5,216.217814,1.143437


In [6]:
co2_chamber = pd.read_csv("../data/interim/co2-linear_model_chamber-wcwh_s21.csv",index_col=0)
co2_chamber.head()

,constant,coefficent
beacon,,
1,251.805181,1.196230
5,314.838208,1.398698
6,15.762631,1.047269
7,74.254982,1.221644
10,-54.402594,1.060180


## PM
Again, two environments.

# Model Comparison

## Carbon Dioxide

## Particulate Matter

## Carbon Monoxide